# Import Libraries - Step 1


In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
path = r'/Users/taraross/Desktop/all/DataAnalytics/Tasks/Achievment4/Instacart Basket Analysis'

In [14]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

### Step 2

In [15]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Step 3

The subset looks different because using the entire dataframe shows all the departments instead of just some of them.  Also the mean looks higher than it was in the subset.

### Step 4

In [16]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/_p/wjzjhv4d3_d8lbx7d_th7l740000gp/T/ipykernel_43035/1681540670.py:1: FutureWarning: The provided callable <function max at 0x112a3cfe0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [17]:
#Check that it added new column
ords_prods_merge.head(15)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Regularly busy,Least busy,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy,Least busy,Average orders,3
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,False,7,0,both,Mid-range product,Regularly busy,Busiest days,Average orders,26
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,False,2,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,9
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.0,False,1,0,both,Mid-range product,Regularly busy,Least busy,Most orders,12
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.0,False,1,0,both,Mid-range product,Regularly busy,Busiest days,Average orders,20
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.0,False,2,1,both,Mid-range product,Regularly busy,Busiest days,Most orders,20


In [18]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [21]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [22]:
#verify data
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head()

,user_id,loyalty_flag,order_number
0,138,Regular customer,28
1,138,Regular customer,30
2,709,New customer,2
3,764,New customer,1
4,764,New customer,3


### Step 5

In [23]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The average price of the loyal customer is actually lower than that of a new or regualr customer.

### Step 6

In [24]:
ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/_p/wjzjhv4d3_d8lbx7d_th7l740000gp/T/ipykernel_43035/72723565.py:1: FutureWarning: The provided callable <function mean at 0x112a3d9e0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [25]:
ords_prods_merge[['user_id', 'mean_product_price']].head(20)

,user_id,mean_product_price
0,138,6.935811
1,138,6.935811
2,709,7.930208
3,764,4.972414
4,764,4.972414
5,777,6.935398
6,825,5.957576
7,910,6.680000
8,1052,7.162500
9,1052,7.162500


In [26]:
#Create spending flag column
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] >= 10, 'spending_flag'] = 'High Spender'

In [27]:
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] < 10, 'spending_flag'] = 'Low Spender'

In [28]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [29]:
#check outcome
ords_prods_merge[['user_id', 'mean_product_price', 'spending_flag']].head(20)

,user_id,mean_product_price,spending_flag
0,138,6.935811,Low Spender
1,138,6.935811,Low Spender
2,709,7.930208,Low Spender
3,764,4.972414,Low Spender
4,764,4.972414,Low Spender
5,777,6.935398,Low Spender
6,825,5.957576,Low Spender
7,910,6.680000,Low Spender
8,1052,7.162500,Low Spender
9,1052,7.162500,Low Spender


### Step 7

In [30]:
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/var/folders/_p/wjzjhv4d3_d8lbx7d_th7l740000gp/T/ipykernel_43035/3824297570.py:1: FutureWarning: The provided callable <function median at 0x112b52840> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [31]:
ords_prods_merge[['user_id', 'median_days']].head(20)

,user_id,median_days
0,138,8.0
1,138,8.0
2,709,8.0
3,764,9.0
4,764,9.0
5,777,11.0
6,825,20.0
7,910,6.0
8,1052,10.0
9,1052,10.0


In [32]:
#Create frequency flag column
ords_prods_merge.loc[ords_prods_merge['median_days'] >20, 'frequency_flag'] = 'Non-frequent customer'

In [33]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] <=20) & (ords_prods_merge['median_days']>10), 'frequency_flag'] = 'Regular customer'

In [34]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <=10, 'frequency_flag'] = 'Frequent customer'

In [35]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

There seems to be missing information, my guess is that it represents customers that have only made one purchase.

In [36]:
# View relevant columns of updated data frame
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days', 'frequency_flag']].head(25)

,user_id,days_since_prior_order,median_days,frequency_flag
0,138,3.0,8.0,Frequent customer
1,138,20.0,8.0,Frequent customer
2,709,6.0,8.0,Frequent customer
3,764,NaN,9.0,Frequent customer
4,764,9.0,9.0,Frequent customer
5,777,26.0,11.0,Regular customer
6,825,30.0,20.0,Regular customer
7,910,30.0,6.0,Frequent customer
8,1052,19.0,10.0,Frequent customer
9,1052,15.0,10.0,Frequent customer


### Step 9 - Export

In [37]:
#This is from 4.9 walk through

In [38]:
ords_prods_merge['prices'].describe()

count    3.240486e+07
mean     1.198023e+01
std      4.956554e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

In [39]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_product_price,spending_flag,median_days,frequency_flag
13100147,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,912404,17,12,2,14,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High Spender,5.0,Frequent customer
13100148,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,603376,17,22,6,16,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High Spender,5.0,Frequent customer
13100149,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,3264360,135,2,2,21,...,High-range product,Regularly busy,Regularly busy,Average orders,4,New customer,1154.792308,High Spender,12.0,Regular customer
13100150,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,892534,135,3,0,8,...,High-range product,Busiest day,Busiest days,Average orders,4,New customer,1154.792308,High Spender,12.0,Regular customer
13100151,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,229704,342,8,1,19,...,High-range product,Regularly busy,Busiest days,Average orders,16,Regular customer,114.426619,High Spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21786876,33664,2 % Reduced Fat Milk,84,16,99999.0,2249946,204099,29,0,8,...,High-range product,Busiest day,Busiest days,Average orders,39,Regular customer,1106.743956,High Spender,4.0,Frequent customer
21786877,33664,2 % Reduced Fat Milk,84,16,99999.0,2363282,204099,31,0,9,...,High-range product,Busiest day,Busiest days,Most orders,39,Regular customer,1106.743956,High Spender,4.0,Frequent customer
21786878,33664,2 % Reduced Fat Milk,84,16,99999.0,3181945,204395,13,3,15,...,High-range product,Regularly busy,Least busy,Most orders,15,Regular customer,451.153540,High Spender,5.0,Frequent customer
21786879,33664,2 % Reduced Fat Milk,84,16,99999.0,2486215,205227,7,3,20,...,High-range product,Regularly busy,Least busy,Average orders,12,Regular customer,1178.381871,High Spender,12.0,Regular customer


In [40]:
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [41]:
ords_prods_merge['prices'].max()

25.0

In [42]:
# Export derived data frame as pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_grouped.pkl'))